In [1]:
# from matplotlib import pyplot as plt
import cv2
import easyocr
import numpy as np
from fpdf import FPDF
from autocorrect import Speller
import preprocessing
from pathlib import Path

In [16]:
# Change img_name to image path
img_path = "sampleImages/f05-352z-02.jpg"

# removing file extension to get image_name
img_name = Path(img_path).stem

# read image into memory
img = cv2.imread(img_path)
# copy image
img_copy = img.copy()
# run preprocessing on the img
prepro_img = preprocessing.preprocess(img)

Couldn't find whiteboard


In [3]:
# display original and processed images
cv2.imshow('img', img)
cv2.imshow('prepro_img', prepro_img)
cv2.waitKey(0)

-1

In [18]:
# loads the readers for each model.
# If using one of the models with a recog network specified you need to follow the steps for using a custom model in the README.md

# base model
# reader_base = easyocr.Reader(['en'])
# model trained on IAM-onDB dataset
reader_wb = easyocr.Reader(['en'], model_storage_directory='model', user_network_directory='user_network', recog_network='wb_v2')
# further training of base model using IAM-onDB dataset and synthetic data generated with TextRecognitionDataGenerator
# reader_g2_wb_bel = easyocr.Reader(['en'], model_storage_directory='model', user_network_directory='user_network', recog_network='g2_wb_bel_v2')


# Loads the spell check
#spell = Speller()

In [19]:
# Use OCR model to extract text
# Uncomment the line with the reader and img you wish to use
# original image
#results = reader_base.readtext(img)
results = reader_wb.readtext(img)
#results = reader_g2_wb_bel.readtext(img)

# preprocessed image
#results = reader_base.readtext(prepro_img)
#results = reader_wb.readtext(prepro_img)
#results = reader_g2_wb_bel.readtext(prepro_img)

In [20]:
# Cleanup Function
def cleanup_text(text):
    # strip out non-ASCII text so we can draw the text on the image
    # using OpenCV
    return "".join([c if ord(c) < 128 else "" for c in text]).strip()

# Duplicate image
cloneImg = img.copy()

# loop over the results and clean up text
for (bbox, text, prob) in results:
    # display the OCR text and associated probability
    print("[INFO] {:.4f}: {}".format(prob, text))
    # unpack the bounding box
    (tl, tr, br, bl) = bbox
    tl = (int(tl[0]), int(tl[1]))
    tr = (int(tr[0]), int(tr[1]))
    br = (int(br[0]), int(br[1]))
    bl = (int(bl[0]), int(bl[1]))
    # cleanup the text and draw the box surrounding the text along
    # with the OCR text itself
    text = cleanup_text(text)
    cv2.rectangle(cloneImg, tl, br, (0, 255, 0), 2)
#	cv2.putText(cloneImg, text, (tl[0], tl[1] - 10),
#   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Uncomment to show image wiht bounding boxes
# cv2.imshow("cloneImg", cloneImg)
# cv2.waitKey(0)

# Print final output text
final_text = ""
for _, text, __ in results:  # _ = bounding box, text = text and __ = confident level
    final_text += " "
    final_text += text
print(final_text)

[INFO] 0.2977: marlrt lar ca wlla lal.
[INFO] 0.1284: eatbtratlebe.
[INFO] 0.4127: p dee na.
 marlrt lar ca wlla lal. eatbtratlebe. p dee na.


In [24]:
# Calculate image size
sz = img.shape

# Convert height and width to inches
H = 8.5*25.4/0.35
W = 11*25.4/0.35

# Scale image to PDF size
# If height is greater than pdf height scale down
if sz[1] > H:
    scale1 = H/sz[1]
else:
    scale1 = 1

# If width is greater than pdf width scale down
if sz[2] > W:
    scale2 = W/sz[2]
else:
    scale2 = 1

# Use larger scale factor
if scale1 < scale2:
    scale = scale1
elif scale2 < scale1:
    scale = scale2
else:
    scale = 1

# Store length of results
i = len(results)

# Define PDF using FPDF
pdf = FPDF(orientation='L', unit='pt')
pdf.add_page()

# Loop through results and save text to PDF
for x in range(i):
    pdf.set_xy(results[x][0][0][0]*scale, results[x][0][0][1]*scale)
    pdf.set_font("Arial", size=16)  # Uncomment for uniform text
    # (results[x][0][2][1] - results[x][0][0][1]-3)*scale*0.8)  # Uncomment for scaled text
    pdf.cell((results[x][0][1][0] - results[x][0][0][0])*scale, (results[x][0][2][1] - results[x][0][0][1])*scale,
             # txt=results[x][1])  # Uncomment for no spell check
             txt=spell(results[x][1]))  # Uncomment for spell check

# Save output PDF
pdf.output(f"sampleResults/{img_name}.pdf")

''